# BIP Framework training example 

In [1]:
using Pkg
Pkg.activate("..")
using BIPs

  Activating project at `~/Documents/WORKS/UBC/RecoBips/BIPs.jl`


In [2]:
using Statistics
using Pkg.Artifacts

Lets begin by bringing in the dataset. It contains tree splits:
* **train**: the training set with 1M jets
* **validation**: the validation set with 400k jets

And of course later we will use the **test** set with other 400k jets to report the results


In [73]:
dataset_path = "/home/jose/Documents/WORKS/UBC/Datalake/reco_ds"

train_data_path = dataset_path*"/train.h5"
val_data_path = dataset_path*"/valid.h5";

### Reading the data

In order to read the datasets, we call the `read_dataset` function:
to read the TopQuark format

In [6]:
train_jets, train_labels = BIPs.read_data("RecoTQ", train_data_path)
print("Number of entries in the training data: ", length(train_jets))

length(targets[1, :]) = 686539
Number of entries in the training data: 

686539

In [74]:
val_jets, val_labels = BIPs.read_data("RecoTQ", val_data_path)
print("Number of entries in the validation data: ", length(val_jets))

length(targets[1, :]) = 196153
Number of entries in the validation data: 

196153

In [9]:
target_scale = maximum(train_labels)

2389.734145552389

Lets examine how one of the jets looks like, each one of the entries is one detected particle's four momentum $(E, p_x, p_y, p_z)$.

However,in order to compute the embeddings, it is necesary to convert the jets to a format that can be used by the framework. The function `data2hyp` allows to convert each detected four momentum to the jet basis, a.k.a $(\tilde p_T, \cos(\theta), \sin(\theta), \tilde y, E_T)$

In [10]:
train_transf_jets = data2basis(train_jets; basis="hyp")
val_transf_jets = data2basis(val_jets; basis="hyp")
println("Transformed jets")

Transformed jets


### The embeddings

Once the jets are converted to the jet basis, it is moment to embed the model using the *Invariant Polynomials*. 

The function `build_ip` allocates efficiently the sparse basis, while the `bip_data` computes the invariant representation of each one of the jets.

In [79]:
f_bip, specs, a_basis = build_ip(order=2, levels=5)
    
function bip_data(dataset_jets)
    storage = zeros(length(dataset_jets), length(specs))
    for i = 1:length(dataset_jets)
        storage[i, :] = f_bip(dataset_jets[i])
    end
    storage[:, 2:end]
end

bip_data (generic function with 1 method)

In [80]:
train_embedded_jets = bip_data(train_transf_jets)
println("Embedded train jets correclty")
val_embedded_jets = bip_data(val_transf_jets)
println("Embedded test jets correclty")

Embedded train jets correclty


Embedded test jets correclty


In [82]:
scaled_train_jets = train_embedded_jets/maximum(train_embedded_jets)
scaled_val_jets = val_embedded_jets/maximum(train_embedded_jets)

scaled_train_labels = train_labels/target_scale
scaled_val_labels = val_labels/target_scale;

### Training a classifier model

The embeddings are now created for the dataset. From this point on, the classification itself is absolutelly versatile. For this specific example we will use the out-of-the box classifier `sklearn.linear_model.HistGradientBoostingClassifier` that bines the data and then applies a grandient boosted trees algorithm. 

Now, lets fit a simple model to the data.


In [83]:
using PyCall
@pyimport sklearn.ensemble as sk_ensemble
@pyimport sklearn.metrics as sk_metrics
@pyimport sklearn.neural_network as sk_nn

In [84]:
GCT = sk_ensemble.HistGradientBoostingRegressor(verbose=true).fit(train_embedded_jets, scaled_train_labels);

1 tree, 31 leaves, max depth = 12, train loss: 44494.07774, val loss: 44704.62935, in 0.071s
Fit 10 trees in 8.597 s, (310 total leaves)
Time spent computing histograms: 0.389s
Time spent finding best splits:  0.035s
Time spent applying splits:      0.054s
Time spent predicting:           0.012s
Binning 0.247 GB of training data: 1.546 s
Binning 0.027 GB of validation data: 0.035 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.00782, val loss: 0.00761, in 0.062s
[2/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.00782, val loss: 0.00761, in 0.044s
[3/100] 1 tree, 31 leaves, max depth = 13, train loss: 0.00782, val loss: 0.00761, in 0.058s
[4/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.00782, val loss: 0.00761, in 0.048s
[5/100] 1 tree, 31 leaves, max depth = 15, train loss: 0.00782, val loss: 0.00761, in 0.050s
[6/100] 1 tree, 31 leaves, max depth = 11, train loss: 0.00782, val loss: 0.00761, in 0.042s
[7/100] 1 tree, 31 leaves

In [87]:
GCT.predict(val_embedded_jets);

# Lest test how we do performance

Now that we understanad the framework, lets see how our model performs on the test set.

In [90]:
test_data_path = dataset_path*"/test.h5"

"/home/jose/Documents/WORKS/UBC/Datalake/reco_ds/test.h5"

In [92]:
test_jets, test_labels = BIPs.read_data("RecoTQ", test_data_path)
test_transf_jets = data2basis(test_jets; basis="hyp")
test_embedded_jets = bip_data(test_transf_jets)
print("Embedded test jets correclty")

length(targets[1, :]) = 98081
Embedded test jets correclty

In [94]:
test_preds = GCT.score(test_embedded_jets, test_labels);

In [95]:
using DelimitedFiles

In [65]:
writedlm( "/home/jose/Documents/WORKS/UBC/RecoBips/saved_basis/train_basis.csv",  train_embedded_jets, ',')
writedlm( "/home/jose/Documents/WORKS/UBC/RecoBips/saved_basis/train_labels.csv",  train_labels, ',')

In [69]:
writedlm( "/home/jose/Documents/WORKS/UBC/RecoBips/saved_basis/val_basis.csv",  val_embedded_jets, ',')
writedlm( "/home/jose/Documents/WORKS/UBC/RecoBips/saved_basis/val_labels.csv",  val_labels, ',')

In [97]:
writedlm( "/home/jose/Documents/WORKS/UBC/RecoBips/saved_basis/test_basis.csv",  test_embedded_jets, ',')
writedlm( "/home/jose/Documents/WORKS/UBC/RecoBips/saved_basis/test_labels.csv",  test_labels, ',')